In [1]:
from flask import Flask, request, jsonify
import requests
import datetime
import pandas as pd
from datetime import datetime, timedelta
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pytz 
from sklearn.preprocessing import StandardScaler
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

def get_stock_data(SYMB, Interval, Range):

    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{SYMB}?region=US&lang=en-US&includePrePost=false&interval={Interval}m&useYfid=true&range={Range}d&corsDomain=finance.yahoo.com&.tsrc=finance'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers, timeout=5)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: Request failed with status code {response.status_code}")
        return None

    # Extract data from the JSON
    meta_data = data['chart']['result'][0]['meta']
    timestamp = data['chart']['result'][0]['timestamp']
    quote_data = data['chart']['result'][0]['indicators']['quote'][0]

    # Convert timestamp to Indian Standard Time (IST)
    utc = pytz.timezone('UTC')
    ist = pytz.timezone('US/Eastern') #US/Eastern Asia/Kolkata
    ist_time = [datetime.fromtimestamp(ts, tz=utc).astimezone(ist) for ts in timestamp]

    # Create a DataFrame from the data
    df = pd.DataFrame({
        'symbol': [meta_data['symbol']] * len(timestamp),
        'Date': ist_time,
        'Open': quote_data['open'],
        'High': quote_data['high'],
        'Low': quote_data['low'],
        'Close': quote_data['close'],
        'Volume': quote_data['volume']
    })
    
    df['Range'] = df['High'] - df['Low']     # High-low
    df['Change'] = df['Close'].diff()  # Calculates the price change between current and previous row's close price
    df['Typical Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['VWAP'] = (df['Typical Price'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    df['SMA20'] = df['Close'].rolling(window=20).mean()
    df['RSI'] = 100 - (100 / (1 + df['Close'].diff(1).apply(lambda x: x if x > 0 else 0).rolling(14).sum() / df['Close'].diff(1).apply(lambda x: abs(x) if x < 0 else 0).rolling(14).sum()))
    df.dropna(inplace=True)
    return df

def next_candle(SYMB, Interval, Range):
    start_time = time.time()

    df = get_stock_data(SYMB,Interval,Range)
    data = df[['Open', 'High', 'Low', 'Close','Volume', 'Range', 'Change', 'VWAP', 'SMA20','RSI']].values
    print(df)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_normalized = scaler.fit_transform(data)
    time_steps = 7

    X, y = [], []
    for i in range(len(data_normalized) - time_steps):
        X.append(data_normalized[i:i+time_steps])
        y.append(data_normalized[i+time_steps])

    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, 10)))
    model.add(LSTM(50))
    model.add(Dense(10))  # 4 output nodes for predicting 'Open', 'High', 'Low', 'Close'

    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    y_pred = model.predict(X_test)

    # Accuracy
    y_pred_original = scaler.inverse_transform(y_pred)
    y_test_original = scaler.inverse_transform(y_test)

    def mape(y_true, y_pred):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    accuracy = 100 - mape(y_test_original, y_pred_original)

    # Prediction
    last_time_step_data = data_normalized[-time_steps:]
    next_future_value = model.predict(np.array([last_time_step_data]))
    next_future_value = scaler.inverse_transform(next_future_value)

    df = df[['Date','Open','High','Low','Close', 'Volume']]
    df['Date'] = pd.to_datetime(df['Date'])
    curr_time = df['Date'].iloc[-1] + timedelta(minutes=int(Interval))
    target = pd.DataFrame(next_future_value, columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Range', 'Change', 'VWAP', 'SMA20','RSI'])
    print(target)
    target['Date'] = curr_time

    end_time = time.time()
    execution_time = end_time - start_time
    return df, target

@app.route('/predict', methods=['POST'])
def predict_next_candle():
    data = request.get_json()
    SYMB = data['SYMB']
    Interval = data['Interval']
    Range = data['Range']
    
    df, target = next_candle(SYMB, Interval, Range)
    
    # Convert DataFrames to JSON
    df_json = df.to_dict(orient='records')
    target_json = target.to_dict(orient='records')
    
    response = {
        "historical_data": df_json,
        "next_candle_prediction": target_json
    }
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, port=5001)


2023-09-26 11:11:06.586399: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:11:06.642115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:11:06.646379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 11:11:07.573441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'flask_cors'

In [1]:
from test import next_candle

df, target = next_candle("AAPL", 1, 2)

2023-11-09 06:57:18.216597: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 06:57:18.272365: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 06:57:18.273265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'typing_extensions'

In [ ]:
target

In [ ]:
import requests
import json

# Define the API endpoint URL
api_url = 'http://localhost:5001/predict'  # Adjust the URL if your Flask app is running on a different port or host

# Define the input data
input_data = {
    "SYMB": "AAPL",     # Replace with the stock symbol you want to predict
    "Interval": 1,     # Replace with the interval (e.g., 15 minutes)
    "Range": 1         # Replace with the range (e.g., 30 days)
}

# Send a POST request to the API
response = requests.post(api_url, json=input_data)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Print the response data (historical data and prediction)
    print(json.dumps(data, indent=4))
else:
    print(f"Error: Request failed with status code {response.status_code}")


{
    "historical_data": [
        {
            "Close": 174.74000549316406,
            "Date": "Mon, 25 Sep 2023 13:49:00 GMT",
            "High": 174.8000030517578,
            "Low": 174.67999267578125,
            "Open": 174.68499755859375,
            "Volume": 124767
        },
        {
            "Close": 174.80810546875,
            "Date": "Mon, 25 Sep 2023 13:50:00 GMT",
            "High": 174.94000244140625,
            "Low": 174.7100067138672,
            "Open": 174.72999572753906,
            "Volume": 164959
        },
        {
            "Close": 174.7510986328125,
            "Date": "Mon, 25 Sep 2023 13:51:00 GMT",
            "High": 174.9499969482422,
            "Low": 174.67999267578125,
            "Open": 174.8000030517578,
            "Volume": 175593
        },
        {
            "Close": 174.9499969482422,
            "Date": "Mon, 25 Sep 2023 13:52:00 GMT",
            "High": 175.0800018310547,
            "Low": 174.75999450683594,
           

In [5]:
from candlestick import candlestick
import pandas as pd
import requests

In [3]:
from candlestick import candlestick
import pandas as pd
import requests

# Find candles where inverted hammer is detected

candles = requests.get('https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d')
candles_dict = candles.json()

candles_df = pd.DataFrame(candles_dict,
                          columns=['T', 'open', 'high', 'low', 'close'])

candles_df['T'] = pd.to_datetime(candles_df['T'], unit='ms')

target = 'InvertedHammers'
candles_df = candlestick.inverted_hammer(candles_df, target=target)
# candles_df = candlestick.doji_star(candles_df)
# candles_df = candlestick.bearish_harami(candles_df)
# candles_df = candlestick.bullish_harami(candles_df)
# candles_df = candlestick.dark_cloud_cover(candles_df)
# candles_df = candlestick.doji(candles_df)
# candles_df = candlestick.dragonfly_doji(candles_df)
# candles_df = candlestick.hanging_man(candles_df)
# candles_df = candlestick.gravestone_doji(candles_df)
# candles_df = candlestick.bearish_engulfing(candles_df)
# candles_df = candlestick.bullish_engulfing(candles_df)
# candles_df = candlestick.hammer(candles_df)
# candles_df = candlestick.morning_star(candles_df)
# candles_df = candlestick.morning_star_doji(candles_df)
# candles_df = candlestick.piercing_pattern(candles_df)
# candles_df = candlestick.rain_drop(candles_df)
# candles_df = candlestick.rain_drop_doji(candles_df)
# candles_df = candlestick.star(candles_df)
# candles_df = candlestick.shooting_star(candles_df)

print(candles_df[candles_df[target] == True][['T', target]])

Exception: InvertedHammer requires at least 1 data

In [ ]:
from candlestick import candlestick
target = 'InvertedHammers'
candles_df = candlestick.inverted_hammer(candles_df, target=target)

In [10]:
import requests
import datetime
import pandas as pd
from datetime import datetime, timedelta
from candlestick import candlestick


def get_stock_data(SYMB, Interval, Range):
    api_url = f"https://api.twelvedata.com/time_series?apikey=7a8e68b26a9d472180f530cbfe1b7dd8&interval={Interval}min&symbol={SYMB}&start_date={(datetime.now() - timedelta(days=Range)).strftime('%Y-%m-%dT%H:%M:%S.%f')}&format=JSON&timezone=US/Eastern"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        print(data)
        time_series = data.get('values', [])
        df = pd.DataFrame(time_series)
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        df = df.iloc[::-1].reset_index(drop=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df.loc[:, df.columns != 'Date'] = df.loc[:, df.columns != 'Date'].apply(pd.to_numeric, errors='coerce')
        # df['Range'] = df['High'] - df['Low']     # High-low
        # df['Change'] = df['Close'].diff()  # Calculates the price change between current and previous row's close price
        # df['Typical Price'] = (df['High'] + df['Low'] + df['Close']) / 3
        # # if df['Volume'].cumsum() != 0:
        # df['VWAP'] = (df['Typical Price'] * df['Volume']).cumsum() / df['Volume'].cumsum()
        # df['SMA20'] = df['Close'].rolling(window=20).mean()
        # df['RSI'] = 100 - (100 / (1 + df['Close'].diff(1).apply(lambda x: x if x > 0 else 0).rolling(14).sum() / df['Close'].diff(1).apply(lambda x: abs(x) if x < 0 else 0).rolling(14).sum()))
        df.dropna(inplace=True)
        from candlestick import candlestick
        target = 'Bullish Doji'
        df = candlestick.dragonfly_doji(df, target=target)
        target = 'bearish Doji'
        df = candlestick.gravestone_doji(df, target=target)
        # print(df.tail())
        return df
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

SYMB = "AAPL"
Interval = 1
Range = 3

df = get_stock_data(SYMB, Interval, Range)

{'meta': {'symbol': 'AAPL', 'interval': '1min', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'mic_code': 'XNGS', 'type': 'Common Stock'}, 'values': [{'datetime': '2023-10-26 15:59:00', 'open': '167.11000', 'high': '167.14500', 'low': '166.72000', 'close': '166.75999', 'volume': '1391527'}, {'datetime': '2023-10-26 15:58:00', 'open': '167.08501', 'high': '167.14500', 'low': '167.06000', 'close': '167.10500', 'volume': '483621'}, {'datetime': '2023-10-26 15:57:00', 'open': '167.02000', 'high': '167.10001', 'low': '167.00000', 'close': '167.08000', 'volume': '396845'}, {'datetime': '2023-10-26 15:56:00', 'open': '167.10500', 'high': '167.13000', 'low': '166.96001', 'close': '167.01500', 'volume': '406886'}, {'datetime': '2023-10-26 15:55:00', 'open': '166.89999', 'high': '167.12000', 'low': '166.87000', 'close': '167.10980', 'volume': '459115'}, {'datetime': '2023-10-26 15:54:00', 'open': '166.85001', 'high': '166.92000', 'low': '166.67999', 'close': '

In [13]:
df[df['Bullish Doji']==True]

,Date,Open,High,Low,Close,Volume,Bullish Doji,bearish Doji
55,2023-10-24 10:25:00,172.78,172.78999,172.5,172.75999,131071,True,False
426,2023-10-25 10:06:00,171.7301,171.735,171.55,171.735,160050,True,False
508,2023-10-25 11:28:00,172.41,172.42999,172.25999,172.42,57053,True,False
1054,2023-10-26 14:04:00,166.825,166.83,166.74001,166.83,88174,True,False
1090,2023-10-26 14:40:00,167.2,167.215,167.09,167.21001,88927,True,False


In [15]:
df[df['bearish Doji']==True]

,Date,Open,High,Low,Close,Volume,Bullish Doji,bearish Doji
529,2023-10-25 11:49:00,172.14,172.25,172.14,172.14999,47030,False,True
770,2023-10-25 15:50:00,171.28,171.42999,171.28,171.285,238219,False,True
1021,2023-10-26 13:31:00,165.82001,165.96001,165.82001,165.83,91825,False,True
1108,2023-10-26 14:58:00,167.52,167.64,167.52,167.5201,144008,False,True
